In [1]:
import torch
import numpy as np
import random
import os
from config import cfg
import argparse
from datasets import make_dataloader
from model import make_model
from processor import do_inference, do_inference_uda
from utils.logger import setup_logger
from utils.metrics import R1_mAP, R1_mAP_eval, R1_mAP_Pseudo, R1_mAP_query_mining, R1_mAP_save_feature, R1_mAP_draw_figure, Class_accuracy_eval
from fair_metrics import *

In [2]:
def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

In [3]:
parser = argparse.ArgumentParser(description="ReID Baseline Training")
parser.add_argument(
    "--config_file", default="configs/uda.yml", help="path to config file", type=str
)

parser.add_argument("opts", help="Modify config options using the command-line", default=None,
                    nargs=argparse.REMAINDER)

parser.add_argument("--local_rank", default=0, type=int)
parser.add_argument('-f')    
args = parser.parse_args()

In [4]:
# python test.py --config_file 
# 'configs/uda.yml' 
# MODEL.DEVICE_ID "('0')" 
# TEST.WEIGHT "('../logs/uda/vit_base/visda/transformer_best_model.pth')" DATASETS.NAMES 'VisDA' DATASETS.NAMES2 'VisDA' OUTPUT_DIR '../logs/uda/vit_base/visda/' DATASETS.ROOT_TRAIN_DIR './data/visda/train/train_image_list.txt' DATASETS.ROOT_TRAIN_DIR2 './data/visda/train/train_image_list.txt' DATASETS.ROOT_TEST_DIR './data/visda/validation/valid_image_list.txt'  


In [5]:
if args.config_file != "":
    cfg.merge_from_file(args.config_file)
cfg.merge_from_list(args.opts)
cfg.freeze()

set_seed(cfg.SOLVER.SEED)
if cfg.MODEL.DIST_TRAIN:
    torch.cuda.set_device(args.local_rank)
else:
    pass

output_dir = cfg.OUTPUT_DIR
if output_dir and not os.path.exists(output_dir):
    os.makedirs(output_dir)

logger = setup_logger("reid_baseline", output_dir, if_train=True)
logger.info("Saving model in the path :{}".format(cfg.OUTPUT_DIR))
logger.info(args)

if args.config_file != "":
    logger.info("Loaded configuration file {}".format(args.config_file))
    with open(args.config_file, 'r') as cf:
        config_str = "\n" + cf.read()
        logger.info(config_str)
logger.info("Running with config:\n{}".format(cfg))

if cfg.MODEL.DIST_TRAIN:
    torch.distributed.init_process_group(backend='nccl', init_method='env://')

os.environ['CUDA_VISIBLE_DEVICES'] = cfg.MODEL.DEVICE_ID



2024-03-06 21:12:26,602 reid_baseline INFO: Saving model in the path :./logs/transoct2slo
2024-03-06 21:12:26,604 reid_baseline INFO: Namespace(config_file='configs/uda.yml', opts=[], local_rank=0, f='/home/shim/.local/share/jupyter/runtime/kernel-b2e1f3e7-fa03-4c8f-9974-a3e10599a1dc.json')
2024-03-06 21:12:26,605 reid_baseline INFO: Loaded configuration file configs/uda.yml
2024-03-06 21:12:26,607 reid_baseline INFO: 
MODEL:
  PRETRAIN_CHOICE: 'pretrain'
  # PRETRAIN_PATH: './.torch/models/jx_vit_base_p16_224-80ecf9dd.pth'
  METRIC_LOSS_TYPE: 'triplet'
  IF_LABELSMOOTH: 'off'
  IF_WITH_CENTER: 'no'
  NAME: 'transformer'
  NO_MARGIN: True
  DEVICE_ID: ('0')
  Transformer_TYPE: 'uda_vit_base_patch16_224_TransReID' #uda_vit_small_patch16_224_TransReID
  STRIDE_SIZE: [16, 16]
  BLOCK_PATTERN: '3_branches'
  TASK_TYPE: 'classify_DA'
  UDA_STAGE: 'UDA'
#  CAMERA_EMBEDDING: True
#  VIEWPOINT_EMBEDDING: True

INPUT:
  SIZE_TRAIN: [256, 256]
  SIZE_TEST: [256, 256]
  SIZE_CROP: [224, 224]
  PR

In [16]:
train_loader, train_loader_normal, val_loader, num_query, num_classes, camera_num, view_num, train_loader1, train_loader2, img_num1, img_num2, s_dataset, t_dataset = make_dataloader(cfg)

model = make_model(cfg, num_class=num_classes, camera_num=camera_num, view_num=view_num)
model.load_param_finetune(cfg.TEST.WEIGHT)

=> Fundus loaded
Dataset statistics:
train oct_train_list and test is slo_test_list
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train   |     2 |     8000 |         1
  test    |     2 |     2000 |         1
  ----------------------------------------
=> Fundus loaded
Dataset statistics:
train slo_test_list and test is slo_test_list
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train   |     2 |     2000 |         1
  test    |     2 |     2000 |         1
  ----------------------------------------
using Transformer_type: uda_vit_base_patch16_224_TransReID as a backbone
using stride: [16, 16], and part number is num_y14 * num_x14
using drop_path_rate is : 0.1
using aie_xishu is : 1.5
using 3branches blocks
make model without initialization
===========building uda transformer===========
Loading pretrained model for finet

In [17]:
do_inference_uda(cfg,
                 model,
                 val_loader,
                 num_query)

2024-03-06 21:17:26,841 reid_baseline.test INFO: Enter inferencing
2024-03-06 21:17:34,827 reid_baseline.test INFO: normal accuracy 0.6315 0.01798652485013008 
2024-03-06 21:17:34,828 reid_baseline.test INFO: Classify Domain Adapatation Validation Results - In the source trained model
2024-03-06 21:17:34,828 reid_baseline.test INFO: Accuracy: 63.1%


In [18]:
folder = '/home/shim/pyspace/CDTrans/data/FairDomain_Classify'

testlist = np.loadtxt('data/FairDomain_Classify/test.txt', dtype=str)

imgattr = {}

for i, f in enumerate(testlist):
    fpath = os.path.join(folder, 'npzs_enhance_224', f)
    data = np.load(fpath)
    
    glaucoma = int(data['glaucoma'])
    race = int(data['race'])
    gender = int(data['gender'])
    ethnicity = int(data['ethnicity'])
    language = int(data['language'])
    maritalstatus = int(data['marriagestatus'])
    attr = [race, gender, ethnicity, language, maritalstatus]
    
    slofname = 'slo_'+f.split('.')[0]+'.jpg'
    
    imgattr[slofname] = attr

In [19]:
device = "cuda"
if cfg.MODEL.TASK_TYPE == 'classify_DA':
    evaluator = Class_accuracy_eval(dataset=cfg.DATASETS.NAMES, logger= logger)
elif cfg.TEST.EVAL:
    evaluator = R1_mAP_eval(num_query, max_rank=50, feat_norm=cfg.TEST.FEAT_NORM)
else:

    evaluator = R1_mAP_draw_figure(cfg, num_query, max_rank=50, feat_norm=True,
                   reranking=cfg.TEST.RE_RANKING)
    # evaluator = R1_mAP_save_feature(num_query, max_rank=50, feat_norm=cfg.TEST.FEAT_NORM,
    #                reranking=cfg.TEST.RE_RANKING)
evaluator.reset()

if device:
    if torch.cuda.device_count() > 1:
        print('Using {} GPUs for inference'.format(torch.cuda.device_count()))
        model = nn.DataParallel(model)
    model.to(device)

model.eval()
img_path_list = []
pred_all = []
gt_all = []
att_all = []
for n_iter, (img, vid, camid, camids, target_view, img_path) in enumerate(val_loader):
    with torch.no_grad():
        
#         print(mms)
        
        img = img.to(device)
        camids = camids.to(device)
        target_view = target_view.to(device)
        target = torch.tensor(vid).to(device)

        if cfg.MODEL.TASK_TYPE == 'classify_DA':
            probs = model(img, img, cam_label=camids, view_label=target_view, return_logits=True)
            
            _, predict = torch.max(probs[1], 1)
            pred = list(np.array(predict.cpu()))
            pred_all.extend(pred)
            gt_all.extend(vid)
            
            # extract attribute
            for pa in img_path:
                att_all.append(imgattr[pa])
            
            evaluator.update((probs[1], vid))
        else:
            feat1, feat2 = model(img, img, cam_label=camids, view_label=target_view, return_logits=False)
            evaluator.update((feat2, vid, camid))
                
if cfg.TEST.EVAL:
    if cfg.MODEL.TASK_TYPE == 'classify_DA':
        accuracy, mean_ent = evaluator.compute()  
        print("Classify Domain Adapatation Validation Results - In the source trained model")
        print("Accuracy: {:.1%}".format(accuracy))
        
    else:
        cmc, mAP, _, _, _, _, _ = evaluator.compute()
        print("Validation Results ")
        print("mAP: {:.1%}".format(mAP))
        for r in [1, 5, 10]:
            print("CMC curve, Rank-{:<3}:{:.1%}".format(r, cmc[r - 1]))
        print(cmc[0], cmc[4])
else:
    print('yes begin saving feature')
    feats, distmats, pids, camids, viewids, img_name_path = evaluator.compute()

    torch.save(feats, os.path.join(cfg.OUTPUT_DIR, 'features.pth'))
    np.save(os.path.join(cfg.OUTPUT_DIR, 'distmat.npy'), distmats)
    np.save(os.path.join(cfg.OUTPUT_DIR, 'label.npy'), pids)
    np.save(os.path.join(cfg.OUTPUT_DIR, 'camera_label.npy'), camids)
    np.save(os.path.join(cfg.OUTPUT_DIR, 'image_name.npy'), img_name_path)
    np.save(os.path.join(cfg.OUTPUT_DIR, 'view_label.npy'), viewids)
    print('over')

2024-03-06 21:17:44,466 reid_baseline INFO: normal accuracy 0.6315 0.01798652485013008 
Classify Domain Adapatation Validation Results - In the source trained model
Accuracy: 63.1%


In [20]:
len(pred_all), len(gt_all), len(att_all)

(2000, 2000, 2000)

In [21]:
pred_all = np.array(pred_all)
gt_all = np.array(gt_all)
att_all = np.array(att_all)

In [22]:
# from sklearn.metrics import confusion_matrix
# matrix = confusion_matrix(gt_all, pred_all)
# acc = matrix.diagonal()/matrix.sum(axis=1) * 100
# aacc = acc.mean() / 100
# aacc

In [23]:
overall_acc, esaccs_by_attrs, overall_auc, esaucs_by_attrs, aucs_by_attrs, dpds, eods, between_group_disparity = evalute_comprehensive_perf_scores(pred_all,
                                                                                                                        gt_all,
                                                                                                                        np.transpose(att_all, (1,0)))

In [24]:
overall_acc, esaccs_by_attrs, overall_auc, esaucs_by_attrs, aucs_by_attrs, dpds, eods, between_group_disparity

(0.6315,
 array([0.59998761, 0.62068389, 0.6128986 , 0.59950154, 0.45559283]),
 0.6361768420967331,
 array([0.60588572, 0.62942677, 0.60674636, 0.60158973, 0.47124818]),
 [array([0.66335045, 0.6188599 , 0.63067261]),
  array([0.63933482, 0.62861066]),
  array([0.63765983, 0.58915441]),
  array([0.62083333, 0.63640492, 0.61507937, 0.61535304]),
  array([0.76682692, 0.65020356, 0.58852665, 0.63423181, 0.66139241,
         0.50568182])],
 array([0.13327454, 0.04552611, 0.10882968, 0.13274911, 0.26702009]),
 array([0.11781748, 0.05791791, 0.15179094, 0.08928571, 0.3125    ]),
 array([[0.0295786 , 0.06993425],
        [0.00842859, 0.01685719],
        [0.03812259, 0.07624518],
        [0.01363366, 0.03352142],
        [0.12375071, 0.41049137]]))

In [ ]:
#baseline
race es_auc: 0.6030806561351596 overall_auc: 0.6327946549490443 aucs_by_attrs: [0.65808729 0.61365157 0.62796002] dpds: 0.1299151754336182 eods: 0.10729116368903913 between_group_disparity: [0.02926702 0.0702214 ] 
gender es_auc: 0.6309805893788262 overall_auc: 0.6327946549490443 aucs_by_attrs: [0.63374986 0.63087487] dpds: 0.034636816929880854 eods: 0.04006863869265742 between_group_disparity: [0.00227166 0.00454333] 
ethnicity es_auc: 0.5996511132768924 overall_auc: 0.6327946549490443 aucs_by_attrs: [0.63462187 0.57935049] dpds: 0.09312995330304003 eods: 0.14477339181286553 between_group_disparity: [0.04367244 0.08734488] 
language es_auc: 0.5888229506705296 overall_auc: 0.6327946549490443 aucs_by_attrs: [0.62083333 0.63449165 0.61507937 0.58949097] dpds: 0.10256410256410264 eods: 0.0892857142857143 between_group_disparity: [0.02578184 0.07111419] 
maritalstatus es_auc: 0.45490739806218333 overall_auc: 0.6327946549490443 aucs_by_attrs: [0.76682692 0.64716096 0.58642581 0.63423181 0.65506329 0.46022727] dpds: 0.2605698529411765 eods: 0.3125 between_group_disparity: [0.144596   0.48451681] 

                        
# race
race es_auc: 0.6018786968218406 overall_auc: 0.6349091550001975 aucs_by_attrs: [0.65641849 0.6032349  0.63321381] dpds: 0.12779565268640236 eods: 0.1283437952679865 between_group_disparity: [0.03428953 0.08376566] 
gender es_auc: 0.6332105633427468 overall_auc: 0.6349091550001975 aucs_by_attrs: [0.63582157 0.63313907] dpds: 0.03686001610723466 eods: 0.04216949587446178 between_group_disparity: [0.00211251 0.00422502] 
ethnicity es_auc: 0.5970130148111232 overall_auc: 0.6349091550001975 aucs_by_attrs: [0.63700565 0.57352941] dpds: 0.10522842543884459 eods: 0.16666666666666663 between_group_disparity: [0.04998844 0.09997688] 
language es_auc: 0.5957686044755565 overall_auc: 0.6349091550001975 aucs_by_attrs: [0.62083333 0.63647176 0.61507937 0.6046798 ] dpds: 0.1102299762093577 eods: 0.0892857142857143 between_group_disparity: [0.01810906 0.05007324] 
maritalstatus es_auc: 0.4638534459975093 overall_auc: 0.6349091550001975 aucs_by_attrs: [0.76682692 0.64857221 0.59190026 0.63908356 0.63623418 0.46022727] dpds: 0.2664292279411765 eods: 0.3125 between_group_disparity: [0.14267154 0.48290318] 
                        
# gender                         


# ethnicity

race es_auc: 0.6058857187101729 overall_auc: 0.6361768420967331 aucs_by_attrs: [0.66335045 0.6188599  0.63067261] dpds: 0.133274542589092 eods: 0.11781747947851284 between_group_disparity: [0.0295786  0.06993425] 
gender es_auc: 0.6294267740446075 overall_auc: 0.6361768420967331 aucs_by_attrs: [0.63933482 0.62861066] dpds: 0.04552610912588817 eods: 0.057917906199132396 between_group_disparity: [0.00842859 0.01685719] 
ethnicity es_auc: 0.6067463577471336 overall_auc: 0.6361768420967331 aucs_by_attrs: [0.63765983 0.58915441] dpds: 0.10882968493693079 eods: 0.15179093567251467 between_group_disparity: [0.03812259 0.07624518] 
language es_auc: 0.6015897257570374 overall_auc: 0.6361768420967331 aucs_by_attrs: [0.62083333 0.63640492 0.61507937 0.61535304] dpds: 0.13274910743265178 eods: 0.0892857142857143 between_group_disparity: [0.01363366 0.03352142] 
maritalstatus es_auc: 0.47124817509444583 overall_auc: 0.6361768420967331 aucs_by_attrs: [0.76682692 0.65020356 0.58852665 0.63423181 0.66139241 0.50568182] dpds: 0.2670200892857143 eods: 0.3125 between_group_disparity: [0.12375071 0.41049137] 


In [25]:
attrs = ['race', 'gender', 'ethnicity', 'language', 'maritalstatus']
for i, attr in enumerate(attrs):
    print(attr, f'es_auc: {esaucs_by_attrs[i]}', f'overall_auc: {overall_auc}', f'aucs_by_attrs: {aucs_by_attrs[i]}',
         f'dpds: {dpds[i]}', f'eods: {eods[i]}', f'between_group_disparity: {between_group_disparity[i]}', '\n')

race es_auc: 0.6058857187101729 overall_auc: 0.6361768420967331 aucs_by_attrs: [0.66335045 0.6188599  0.63067261] dpds: 0.133274542589092 eods: 0.11781747947851284 between_group_disparity: [0.0295786  0.06993425] 

gender es_auc: 0.6294267740446075 overall_auc: 0.6361768420967331 aucs_by_attrs: [0.63933482 0.62861066] dpds: 0.04552610912588817 eods: 0.057917906199132396 between_group_disparity: [0.00842859 0.01685719] 

ethnicity es_auc: 0.6067463577471336 overall_auc: 0.6361768420967331 aucs_by_attrs: [0.63765983 0.58915441] dpds: 0.10882968493693079 eods: 0.15179093567251467 between_group_disparity: [0.03812259 0.07624518] 

language es_auc: 0.6015897257570374 overall_auc: 0.6361768420967331 aucs_by_attrs: [0.62083333 0.63640492 0.61507937 0.61535304] dpds: 0.13274910743265178 eods: 0.0892857142857143 between_group_disparity: [0.01363366 0.03352142] 

maritalstatus es_auc: 0.47124817509444583 overall_auc: 0.6361768420967331 aucs_by_attrs: [0.76682692 0.65020356 0.58852665 0.63423181 

In [10]:
auc = 0.6327946549
group_aucs_race = [0.67913992, 0.61897936, 0.6431148]
mean_disparity = float(np.std(group_aucs_race) / (auc))
max_disparity = (np.max(group_aucs_race)-np.min(group_aucs_race)) / np.abs(auc)
print(mean_disparity, max_disparity)

auc = 0.6327946549
group_aucs_race = [0.64704352, 0.6448687]
mean_disparity = float(np.std(group_aucs_race) / (auc))
max_disparity = (np.max(group_aucs_race)-np.min(group_aucs_race)) / np.abs(auc)
print(mean_disparity, max_disparity)

auc = 0.6327946549
group_aucs_race = [0.64830508, 0.58915441]
mean_disparity = float(np.std(group_aucs_race) / (auc))
max_disparity = (np.max(group_aucs_race)-np.min(group_aucs_race)) / np.abs(auc)
print(mean_disparity, max_disparity)

0.03825089118445411 0.09309112395227297
0.0016826342556808332 0.0033652685113616664
0.045764221217604914 0.09152844243520983


In [11]:
auc = 0.7615
group_aucs_race = [0.7673940949935816, 0.751624617737003, 0.7564252346931267]
mean_disparity = float(np.std(group_aucs_race) / (auc))
max_disparity = (np.max(group_aucs_race)-np.min(group_aucs_race)) / np.abs(auc)
print(mean_disparity, max_disparity)

auc = 0.7615
group_aucs_race = [0.7632310259898939, 0.7594721329046088]
mean_disparity = float(np.std(group_aucs_race) / (auc))
max_disparity = (np.max(group_aucs_race)-np.min(group_aucs_race)) / np.abs(auc)
print(mean_disparity, max_disparity)

auc = 0.7615
group_aucs_race = [0.7630059140317837, 0.6954656862745098]
mean_disparity = float(np.std(group_aucs_race) / (auc))
max_disparity = (np.max(group_aucs_race)-np.min(group_aucs_race)) / np.abs(auc)
print(mean_disparity, max_disparity)

0.008667084860357523 0.020708440258146586
0.0024680847572456322 0.0049361695144912645
0.0443468337211254 0.0886936674422508


In [5]:
auc = 0.6724
group_aucs_race = [0.6681643132220796, 0.6770355504587156, 0.6726104910654197]
mean_disparity = float(np.std(group_aucs_race) / (auc))
max_disparity = (np.max(group_aucs_race)-np.min(group_aucs_race)) / np.abs(auc)
print(mean_disparity, max_disparity)

auc = 0.6724
group_aucs_race = [0.6705647010757496, 0.6780412647374062]
mean_disparity = float(np.std(group_aucs_race) / (auc))
max_disparity = (np.max(group_aucs_race)-np.min(group_aucs_race)) / np.abs(auc)
print(mean_disparity, max_disparity)

auc = 0.6724
group_aucs_race = [0.6766885827248599, 0.5575980392156863]
mean_disparity = float(np.std(group_aucs_race) / (auc))
max_disparity = (np.max(group_aucs_race)-np.min(group_aucs_race)) / np.abs(auc)
print(mean_disparity, max_disparity)

0.005386185093454525 0.013193392677923802
0.005559610099387723 0.011119220198775447
0.08855632325191373 0.17711264650382746


In [4]:
auc = 0.6963
group_aucs_race = [0.6785622593068037, 0.6929950305810397, 0.6803561269666935]
mean_disparity = float(np.std(group_aucs_race) / (auc))
max_disparity = (np.max(group_aucs_race)-np.min(group_aucs_race)) / np.abs(auc)
print(mean_disparity, max_disparity)

auc = 0.6754
group_aucs_race = [0.6722162908031312, 0.6839496248660236]
mean_disparity = float(np.std(group_aucs_race) / (auc))
max_disparity = (np.max(group_aucs_race)-np.min(group_aucs_race)) / np.abs(auc)
print(mean_disparity, max_disparity)

auc = 0.6754
group_aucs_race = [0.6801433904912941, 0.5520833333333334]
mean_disparity = float(np.std(group_aucs_race) / (auc))
max_disparity = (np.max(group_aucs_race)-np.min(group_aucs_race)) / np.abs(auc)
print(mean_disparity, max_disparity)

0.009224104155145327 0.020727805937435062
0.008686211180702142 0.017372422361404283
0.09480312197065499 0.18960624394130998


In [6]:
auc = 0.6758
group_aucs_race = [0.7037227214377406, 0.6890290519877675, 0.676675334743915]
mean_disparity = float(np.std(group_aucs_race) / (auc))
max_disparity = (np.max(group_aucs_race)-np.min(group_aucs_race)) / np.abs(auc)
print(mean_disparity, max_disparity)

auc = 0.6758
group_aucs_race = [0.6783851807883691, 0.6701902465166131]
mean_disparity = float(np.std(group_aucs_race) / (auc))
max_disparity = (np.max(group_aucs_race)-np.min(group_aucs_race)) / np.abs(auc)
print(mean_disparity, max_disparity)

auc = 0.6758
group_aucs_race = [0.6798746709837997, 0.579656862745098]
mean_disparity = float(np.std(group_aucs_race) / (auc))
max_disparity = (np.max(group_aucs_race)-np.min(group_aucs_race)) / np.abs(auc)
print(mean_disparity, max_disparity)

0.01635959581708581 0.04002276811752837
0.006063135744122517 0.012126271488245034
0.07414753495020845 0.1482950699004169
